In [1]:
from prophet import Prophet
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, add_changepoints_to_plot
# from keras.models import Sequential
# from keras.layers import Dense

# from keras.optimizers import Adam
import prophet
warnings.filterwarnings('ignore')

/Users/amandatsuetaki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/amandatsuetaki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
data=pd.read_csv('univariate_models/multivar.csv', index_col=0, parse_dates=[0])
data= data.set_index('date')
print(data.shape)
data.head()

(2023, 52)


,iso_code,continent,location,total_cases,new_cases,new_deaths,total_cases_per_million,new_cases_per_million,icu_patients_per_million,hosp_patients_per_million,...,new_deaths_4_week_min,year,season,comparison_with_previous,icu_utilization_ratio,healthcare_strain_index,cumulative_cases_threshold,gdp_stringency_interaction,Holiday,Normal day
date,,,,,,,,,,,,,,,,,,,,,
2020-03-01,BEL,Europe,Belgium,1.0,0.0,0.0,0.086,0.000,NaN,NaN,...,NaN,2020,Spring,NaN,NaN,NaN,0,4.739368e+05,0,1
2020-03-08,BEL,Europe,Belgium,316.0,315.0,0.0,27.111,27.025,0.000,0.000,...,NaN,2020,Spring,0.0,0.000000,0.000000,1,5.925276e+05,0,1
2020-03-15,BEL,Europe,Belgium,1362.0,1046.0,9.0,116.850,89.740,4.547,22.564,...,NaN,2020,Spring,9.0,0.080621,4.806915,2,2.172601e+06,0,1
2020-03-22,BEL,Europe,Belgium,4473.0,3111.0,140.0,383.753,266.903,27.969,142.503,...,0.0,2020,Spring,131.0,0.495904,30.225532,3,3.475821e+06,0,1
2020-03-29,BEL,Europe,Belgium,12031.0,7558.0,602.0,1032.179,648.426,82.447,389.158,...,0.0,2020,Spring,462.0,1.461826,83.617908,4,3.475821e+06,0,1


In [ ]:
def process(df):
    lag1df = df.shift(1)
    lag1df.columns = ['lag1_' + str(col) for col in lag1df.columns]
    df=df[['new_deaths']].join(lag1df)
    return df.fillna(0) 
data=process(data)

#data = pd.get_dummies(data, columns=['location'])
train_size = int(len(data) * 0.8)
train=data.iloc[:train_size ] 
test=data.iloc[train_size: ] 
print(train.shape, test.shape)


: 

In [3]:
model = prophet.Prophet()
for col in train.columns:
    if col not in ['date', 'total_purchase_amt']:
        model.add_regressor(col)
model=model.fit(train.reset_index().rename(columns={'date':'ds', 'new_deaths':'y'}))

NameError: name 'train' is not defined